## Predicting the price for an Airbnb Host in Berlin

Airbnb has successfully disrupted the traditional hospitality industry as more and more travelers decide to use Airbnb as their primary accommodation provider. Since its inception in 2008, Airbnb has seen an enormous growth, with the number of rentals listed on its website growing exponentially each year.

In Germany, no city is more popular than Berlin. That implies that Berlin is one of the hottest markets for Airbnb in Europe, with over 22,552 listings as of November 2018. With a size of 891 km², this means there are roughly 25 homes being rented out per km² in Berlin on Airbnb!

Considering the possibility that I might have to relocate for a new data science job, but want to keep my current flat in Berlin (which is quite cheap!), I might wonder if it could be worth it to offer my jewel on Airbnb. Could this perhaps be a profitable option? However, it is difficult for potential hosts to know what the true value of their home is, and how in-demand their home might be. And since location and furniture are obviously fixed for the most part, is there anything else a host can influence - such as description, communication patterns, and/or additional services to boost their earnings?

The following question will drive this project:

> **Can we determine a fairly spot-on daily price for a new accommodation that fits into its specific market environment and competitors in Berlin?** <br>

The question focuses on the accommodation features and decisions a new host can make with regards to initial presentation, i.e. posting a picture of him- or herself on the website, determining a minimum length of stay, offering instant bookings etc. A machine learning algorithm will be applied to try to get an answer. 

### The dataset

In the first notebook, I will perform an analysis of the detailed Berlin listings data, sourced from the Inside Airbnb website, in order to understand the rental landscape and try to recommend a price for a newbie entering the market. The dataset is named `listings.csv.gz` and was scraped on November 07th 2018.

## Table of Contents
<a id='Table of contents'></a>

### <a href='#1. Obtaining and Viewing the Data'> 1. Obtaining and Viewing the Data </a>

### <a href='#2. Preprocessing the Data'> 2. Preprocessing the Data </a>
* <a href='#2.1. Deciding which columns to work with'> 2.1. Deciding which columns to work with </a>
* <a href='#2.2. Cleaning Price Columns'> 2.2. Cleaning Price Columns </a>
* <a href='#2.3. Dealing with Missing Values'> 2.3. Dealing with Missing Values </a>
* <a href='#2.4. Feature Engineering 1: Distance to Centroid of Berlin'> 2.4. Feature Engineering 1: Distance to Centroid of Berlin </a>
* <a href='#2.5. Feature Engineering 2: Lodging Size'> 2.5. Feature Engineering 2: Lodging Size </a>
* <a href='#2.6. Feature Engineering 3: Lodging Amenities'> 2.6. Feature Engineering 3: Lodging Amenities </a>

### <a href='#3. Exploratory Data Analysis (EDA)'> 3. Exploratory Data Analysis (EDA) </a>

### <a href='#4. Modeling the Data'> 4. Modeling the Data </a>
* <a href='#4.1. Preparing Target and Features'> 4.1. Preparing Target and Features </a>
* <a href='#4.2. Splitting and Scaling the Data'> 4.2. Splitting and Scaling the Data </a>
* <a href='#4.3. Training an XGBoost Regressor'> 4.3. Training an XGBoost Regressor </a>
* <a href='#4.4. Cross Validation'> 4.4. Cross Validation </a>

### <a href='#5. Interpreting the Data'> 5. Interpreting the Data </a>

### <a href='#6. Appendix'> 6. Appendix </a>

### 1. Obtaining and Viewing the Data 
<a id='1. Obtaining and Viewing the Data'></a>

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns

import re
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_initial = pd.read_csv('../input/listings_summary.csv')

# checking shape
print("The dataset has {} rows and {} columns.".format(*df_initial.shape))

# ... and duplicates
print("It contains {} duplicates.".format(df_initial.duplicated().sum()))

In [ ]:
df_initial.head(1)

In [ ]:
# check the columns we currently have
df_initial.columns

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 2. Preprocessing the Data 
<a id='2. Preprocessing the Data'></a>

#### 2.1. Deciding which columns to work with 
<a id='2.1. Deciding which columns to work with'></a>

Let's imagine we are in the shoes of someone who'd like to offer their home. Fixed features of our property include its rooms, size, and location. We also can decide on how we want to be listed: with a picture or not, how many minimum nights we want a guest to stay, whether we are instantly bookable, how we handle cancellations, etc. But we can neither be a "super host", nor do we have any reviews yet to show - although they can be very important for setting a price. So, let's focus only on features we can influence:

In [ ]:
# define the columns we want to keep
columns_to_keep = ['id', 'space', 'description', 'host_has_profile_pic', 'neighbourhood_group_cleansed', 
                   'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',  
                   'bedrooms', 'bed_type', 'amenities', 'square_feet', 'price', 'cleaning_fee', 
                   'security_deposit', 'extra_people', 'guests_included', 'minimum_nights',  
                   'instant_bookable', 'is_business_travel_ready', 'cancellation_policy']

df_raw = df_initial[columns_to_keep].set_index('id')
print("The dataset has {} rows and {} columns - after dropping irrelevant columns.".format(*df_raw.shape))

By the way, how many different **room types** do we have?

In [ ]:
df_raw.room_type.value_counts(normalize=True)

And how many different **property types** are we up against?

In [ ]:
df_raw.property_type.value_counts(normalize=True)

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 2.2. Cleaning Price Columns
<a id='2.2. Cleaning Price Columns'></a>

In [ ]:
df_raw[['price', 'cleaning_fee', 'extra_people', 'security_deposit']].head(3)

Let's first check if there are any null values in the `price` column:

In [ ]:
# checking Nan's in "price" column
df_raw.price.isna().sum()

No, so we don't need to drop any rows. And what about the `cleaning_fee`?

In [ ]:
# Nan's in "cleaning_fee" column
df_raw.cleaning_fee.isna().sum()

There are plenty of Nan's. It's more than likely that these hosts do not charge any extra cleaning fee. So let's simply replace these null values with $0.00:

In [ ]:
df_raw.cleaning_fee.fillna('$0.00', inplace=True)
df_raw.cleaning_fee.isna().sum()

The same is true for the `security_deposit`:

In [ ]:
df_raw.security_deposit.isna().sum()

In [ ]:
df_raw.security_deposit.fillna('$0.00', inplace=True)
df_raw.security_deposit.isna().sum()

In [ ]:
df_raw.extra_people.isna().sum()

Let's remove the dollar signs in all four columns and convert the string values into numerical ones:

In [ ]:
# clean up the columns (by method chaining)
df_raw.price = df_raw.price.str.replace('$', '').str.replace(',', '').astype(float)
df_raw.cleaning_fee = df_raw.cleaning_fee.str.replace('$', '').str.replace(',', '').astype(float)
df_raw.security_deposit = df_raw.security_deposit.str.replace('$', '').str.replace(',', '').astype(float)
df_raw.extra_people = df_raw.extra_people.str.replace('$', '').str.replace(',', '').astype(float)

We shouldn't miss investigating the `price` - it might need some cleaning to be of use to us:

In [ ]:
df_raw['price'].describe()

In [ ]:
red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')
df_raw['price'].plot(kind='box', xlim=(0, 1000), vert=False, flierprops=red_square, figsize=(16,2));

A bit messy, as expected! 

75% of the apartments charge up to 68€ - but the maximum value is 9000€. Let's decide on a limit of 400€, after which the outliers seem to "fringe", and drop all records that charge more than that. Oddly enough, we have prices set to zero. So let's drop those records, too:

In [ ]:
df_raw.drop(df_raw[ (df_raw.price > 400) | (df_raw.price == 0) ].index, axis=0, inplace=True)

In [ ]:
df_raw['price'].describe()

In [ ]:
print("The dataset has {} rows and {} columns - after being price-wise preprocessed.".format(*df_raw.shape))

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 2.3. Dealing with Missing Values
<a id='2.3. Dealing with Missing Values'></a>

In [ ]:
df_raw.isna().sum()

In [ ]:
# drop columns with too many Nan's
df_raw.drop(columns=['square_feet', 'space'], inplace=True)

In [ ]:
# drop rows with NaN's in bathrooms and bedrooms
df_raw.dropna(subset=['bathrooms', 'bedrooms', ], inplace=True)

In [ ]:
df_raw.host_has_profile_pic.unique()

In [ ]:
# replace host_has_profile_pic Nan's with no
df_raw.host_has_profile_pic.fillna(value='f', inplace=True)
df_raw.host_has_profile_pic.unique()

In [ ]:
df_raw.isna().sum()

In [ ]:
print("The dataset has {} rows and {} columns - after having dealt with missing values.".format(*df_raw.shape))

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 2.4. Feature Engineering 1: Distance to Centroid of Berlin
<a id='2.4. Feature Engineering 1: Distance to Centroid of Berlin'></a>

Location is always an important factor in lodging services. To make it more descriptive, I decided to calculate each accommodation's distance to the so-called centroid of Berlin instead of just relying on the neighbourhoods or areas. 

For our convenience, let's write a quick function that does this, apply it to each accommodation, and store the values in a new column:

In [ ]:
from geopy.distance import great_circle

In [ ]:
def distance_to_mid(lat, lon):
    berlin_centre = (52.5027778, 13.404166666666667)
    accommodation = (lat, lon)
    return great_circle(berlin_centre, accommodation).km

In [ ]:
df_raw['distance'] = df_raw.apply(lambda x: distance_to_mid(x.latitude, x.longitude), axis=1)

In [ ]:
df_raw.head(2)

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 2.5. Feature Engineering 2: Lodging Size
<a id='2.5. Feature Engineering 2: Lodging Size'></a>

One of the most important pieces of information for predicting the rate is the size. Since the column `square_feet` was heavily filled with null values, we dropped it in the previous section. (Besides, size in Germany is expressed in square meters, not in square feet.)

Let's check, if the column `description` reveals any information about size instead: 

In [ ]:
#list(df_raw.description[:10])

In [ ]:
df_raw.description.isna().sum()

The `description` column seems to be rich in content. Let's extract 
- all double-digit or three-digit numbers 
- that are followed by one of the two characters "s" or "m" (covering "sqm", "square meters", "m2" etc.) and 
- may or may not be connected by white space. 

Single- or more than three-digit numbers for accommodation sizes are quite unlikely.

I know, it's a bold move - but let's give it a try...

***Extracting size from text***

In [ ]:
# extract numbers 
df_raw['size'] = df_raw['description'].str.extract('(\d{2,3}\s?[smSM])', expand=True)
df_raw['size'] = df_raw['size'].str.replace("\D", "")

# change datatype of size into float
df_raw['size'] = df_raw['size'].astype(float)

print('NaNs in size_column absolute:     ', df_raw['size'].isna().sum())
print('NaNs in size_column in percentage:', round(df_raw['size'].isna().sum()/len(df_raw),3), '%')

In [ ]:
df_raw[['description', 'size']].head(10)

In [ ]:
#list(df_raw.description[:10])

Comparing the results, we did a pretty good job with the first 4 records, but filtered an incorrect number for the last record. Okay, let's keep that in mind: there may be mistakes in the size we engineered from the text! 

In [ ]:
# drop description column
df_raw.drop(['description'], axis=1, inplace=True)

***Predicting missing values with regression***

As calculated further up, half of our records still don't have a size. That means we have a problem! Dropping these records isn't an option as we would loose too much valuable information. Simply replacing it with the mean or median makes no sense. That leaves a third option: predict the missing value with a Machine Learning Algorithm. To not make it too complicated, we'll only use numerical features. Next, we have to split our data into 
- a) a training set where we have sizes and 
- b) a test set where we don't.

In [ ]:
df_raw.info()

In [ ]:
# filter out sub_df to work with
sub_df = df_raw[['accommodates', 'bathrooms', 'bedrooms',  'price', 'cleaning_fee', 
                 'security_deposit', 'extra_people', 'guests_included', 'distance', 'size']]

In [ ]:
# split datasets
train_data = sub_df[sub_df['size'].notnull()]
test_data  = sub_df[sub_df['size'].isnull()]

# define X
X_train = train_data.drop('size', axis=1)
X_test  = test_data.drop('size', axis=1)

# define y
y_train = train_data['size']

In [ ]:
print("Shape of Training Data:", train_data.shape)
print("Shape of Test Data:    ",test_data.shape)
print("\nShape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("\nShape of y_train:", y_train.shape)

In [ ]:
# import Linear Regression
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit model to training data
linreg.fit(X_train, y_train)

In [ ]:
# making predictions
y_test = linreg.predict(X_test)

Now that we have predicted the missing sizes, let's cast `y_test` into a dataframe with a column `size`:

In [ ]:
y_test = pd.DataFrame(y_test)
y_test.columns = ['size']
print(y_test.shape)
y_test.head()

In [ ]:
print(X_test.shape)
X_test.head()

To combine `y_test` and `X_test` back to our full test_dataframe, we have to create a bit of a circuit. As we can see here, the `X_test` dataframe has a specific index that's not compatible with `y_test`, where the numbers simply start from 0 onwards. So we need to provide `y_test` with the `X_test` index to make sure everyting fits together.

In [ ]:
# make the index of X_test to an own dataframe
prelim_index = pd.DataFrame(X_test.index)
prelim_index.columns = ['prelim']

# ... and concat this dataframe with y_test
y_test = pd.concat([y_test, prelim_index], axis=1)
y_test.set_index(['prelim'], inplace=True)
y_test.head()

In [ ]:
new_test_data = pd.concat([X_test, y_test], axis=1)

In [ ]:
print(new_test_data.shape)
new_test_data.head()

In [ ]:
new_test_data['size'].isna().sum()

Once we have completed our new train_data, we stack test_data and train_data back to a new sub_dataframe across the rows:

In [ ]:
# combine train and test data back to a new sub df
sub_df_new = pd.concat([new_test_data, train_data], axis=0)

print(sub_df_new.shape)
sub_df_new.head()

In [ ]:
sub_df_new['size'].isna().sum()

Finally, the last big step: merging our sub_dataframe with the remaining columns from our original dataframe - this time, across the columns:

In [ ]:
# prepare the multiple columns before concatening
df_raw.drop(['accommodates', 'bathrooms', 'bedrooms', 'price', 'cleaning_fee', 
             'security_deposit', 'extra_people', 'guests_included', 'distance', 'size'], 
            axis=1, inplace=True)

In [ ]:
# concate back to complete dataframe
df = pd.concat([sub_df_new, df_raw], axis=1)

print(df.shape)
df.head(2)

***Investigating sizes***

Let's thoroughly examine our new `size` column:

In [ ]:
df['size'].isna().sum()

In [ ]:
df['size'].describe()

In [ ]:
red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')
df['size'].plot(kind='box', xlim=(0, 1000), vert=False, flierprops=red_square, figsize=(16,2));

To be on the safe side, let’s remove all outliers over 300 square meters, and all those with 0:

In [ ]:
df.drop(df[ (df['size'] == 0.) | (df['size'] > 300.) ].index, axis=0, inplace=True)

In [ ]:
print("The dataset has {} rows and {} columns - after being engineered.".format(*df.shape))

*Back to: <a href='#Table of contents'> Table of contents</a>*
#### 2.6. Feature Engineering 3: Lodging Amenities
<a id='2.6. Feature Engineering 3: Lodging Amenities'></a>

I'm interested in what amenities hosts offer their guests, and in order to enrich our prediction, whether we can determine what some of the more special and/or rare amenities might be that make a property more desirable.

In [ ]:
from collections import Counter

In [ ]:
results = Counter()
df['amenities'].str.strip('{}')\
               .str.replace('"', '')\
               .str.lstrip('\"')\
               .str.rstrip('\"')\
               .str.split(',')\
               .apply(results.update)

results.most_common(30)

In [ ]:
# create a new dataframe
sub_df = pd.DataFrame(results.most_common(30), columns=['amenity', 'count'])

In [ ]:
# plot the Top 20
sub_df.sort_values(by=['count'], ascending=True).plot(kind='barh', x='amenity', y='count',  
                                                      figsize=(10,7), legend=False, color='darkgrey',
                                                      title='Amenities')
plt.xlabel('Count');

Let's add columns with amenities that are somewhat unique and not offered by all hosts: 
- a laptop-friendly workspace
- a TV
- kid friendly accommodation 
- smoker friendly and 
- being greeted by the host.

After doing this, let's drop the original column:

In [ ]:
df['Laptop_friendly_workspace'] = df['amenities'].str.contains('Laptop friendly workspace')
df['TV'] = df['amenities'].str.contains('TV')
df['Family_kid_friendly'] = df['amenities'].str.contains('Family/kid friendly')
df['Host_greets_you'] = df['amenities'].str.contains('Host greets you')
df['Smoking_allowed'] = df['amenities'].str.contains('Smoking allowed')

In [ ]:
df.drop(['amenities'], axis=1, inplace=True)

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 3. Exploratory Data Analysis (EDA)
<a id='3. Exploratory Data Analysis (EDA)' ></a>

**> Price Differences on a Map**

In [ ]:
df.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, figsize=(10,7), 
        c="price", cmap="gist_heat_r", colorbar=True, sharex=False);

**> Price Differences by Neighbourhood**

In [ ]:
df['neighbourhood_group_cleansed'].value_counts().sort_values().plot(kind='barh', color='darkgrey')
plt.title('Number of Accommodations per District');

In [ ]:
# group_by neighbourhood groups, take the median price and store new values in sub_df 
df_grouped = pd.DataFrame(df.groupby(['neighbourhood_group_cleansed'])['price'].agg(np.median))
df_grouped.reset_index(inplace=True)

# plot this 
df_grouped.sort_values(by=['price'], ascending=True)\
          .plot(kind='barh', x='neighbourhood_group_cleansed', y='price', 
                figsize=(10,6), legend=False, color='salmon')

plt.xlabel('\nMedian Price', fontsize=12)
plt.ylabel('District\n', fontsize=12)
plt.title('\nMedian Prices by Neighbourhood\n', fontsize=14, fontweight='bold');

In [ ]:
red_square = dict(markerfacecolor='salmon', markeredgecolor='salmon', marker='.')

df.boxplot(column='price', by='neighbourhood_group_cleansed', 
           flierprops=red_square, vert=False, figsize=(10,8))

plt.xlabel('\nMedian Price', fontsize=12)
plt.ylabel('District\n', fontsize=12)
plt.title('\nBoxplot: Prices by Neighbourhood\n', fontsize=14, fontweight='bold')

# get rid of automatic boxplot title
plt.suptitle('');

**> Price Differences by Accommodation Distance to Center of Berlin**

In [ ]:
df.plot.scatter(x="distance", y="price", figsize=(9,6), c='dimgrey')
plt.title('\nRelation between Distance & Median Price\n', fontsize=14, fontweight='bold');

In [ ]:
sns.jointplot(x=df["distance"], y=df["price"], kind='hex')
plt.title('\nRelation between Distance & Median Price\n', fontsize=14, fontweight='bold');

**> Price Differences by Accommodation Size**

In [ ]:
sns.set_style("white")
cmap = sns.cubehelix_palette(rot=-.2, as_cmap=True)

fig, ax = plt.subplots(figsize=(12,7))
ax = sns.scatterplot(x="size", y="price", size='cleaning_fee', sizes=(5, 200),
                      hue='size', palette=cmap,  data=df)

plt.title('\nRelation between Size & Median Price\n', fontsize=14, fontweight='bold')

# putting legend out of the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

**> Price Differences by Number of Bedrooms**

In [ ]:
plt.figure(figsize=(6,6))
sns.heatmap(df.groupby(['neighbourhood_group_cleansed', 'bedrooms']).price.median().unstack(), 
            cmap='Reds', annot=True, fmt=".0f")

plt.xlabel('\nBedrooms', fontsize=12)
plt.ylabel('District\n', fontsize=12)
plt.title('\nHeatmap: Median Prices by Neighbourhood and Number of Bedrooms\n\n', fontsize=14, fontweight='bold');

**> Price Differences by Cancellation Policy**

In [ ]:
# group_by neighbourhood groups, take the median price and store new values in sub_df 
df_grouped = pd.DataFrame(df.groupby(['cancellation_policy'])['price'].agg(np.median))
df_grouped.reset_index(inplace=True)

# plot this 
df_grouped.sort_values(by=['price'], ascending=True)\
          .plot(kind='barh', x='cancellation_policy', y='price', 
                figsize=(9,5), legend=False, color='darkblue')

plt.xlabel('\nMedian Price', fontsize=12)
plt.ylabel('Cancellation Policy\n', fontsize=12)
plt.title('\nMedian Prices by Cancellation Policy\n', fontsize=14, fontweight='bold');

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 4. Modeling the Data 
<a id='4. Modeling the Data'></a>

#### 4.1. Preparing Target and Features
<a id='4.1. Preparing Target and Features'></a>

In [ ]:
df.columns

In [ ]:
df.info()

As we work with the distance to the center, let's drop the `neighbourhood_group_cleansed`. Furthermore, `property_types` may not be that helpful.

In [ ]:
df.drop(['latitude', 'longitude', 'neighbourhood_group_cleansed', 'property_type'], axis=1, inplace=True)

Now let's convert all string columns into categorical ones:

In [ ]:
for col in ['host_has_profile_pic', 'room_type', 'bed_type', 'instant_bookable', 
            'is_business_travel_ready', 'cancellation_policy']:
    df[col] = df[col].astype('category')

In [ ]:
# define our target
target = df[["price"]]

# define our features 
features = df.drop(["price"], axis=1)

***Recoding Categorical Features***

Machine learning algorithms generally need all data - including categorical data - in numeric form. To satisfy these algorithms, categorical features are converted into separate binary features called dummy variables. Therefore, we have to find a way to represent these variables as numbers before handing them off to the model. One typical way of doing this in one-hot encoding, which creates a new column for each unique category in a categorical variable. Each observation receives a 1 in the column for its corresponding category (= "HOT") and a 0 in all other new columns. To conduct one-hot encoding, we use the pandas get_dummies function.

In [ ]:
num_feats = features.select_dtypes(include=['float64', 'int64', 'bool']).copy()

# one-hot encoding of categorical features
cat_feats = features.select_dtypes(include=['category']).copy()
cat_feats = pd.get_dummies(cat_feats)

In [ ]:
features_recoded = pd.concat([num_feats, cat_feats], axis=1)

In [ ]:
print(features_recoded.shape)
features_recoded.head(2)

#### 4.2. Splitting and Scaling the Data
<a id='4.2. Splitting and Scaling the Data'></a>

In [ ]:
# import train_test_split function
from sklearn.model_selection import train_test_split
# import metrics
from sklearn.metrics import mean_squared_error, r2_score

# split our data
X_train, X_test, y_train, y_test = train_test_split(features_recoded, target, test_size=0.2)

One of the challenges in building models is mixing features that have different scales. Look at our dataset and compare bathrooms with size or maximum_nights. When we mix units with ranges that have different orders of magnitude, our models may not be able to find the proper coefficients. To account for this problem, we standardize or normalize the features.

In [ ]:
# scale data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test  = sc.transform(X_test)

#### 4.3. Training an XGBoost Regressor
<a id='4.3. Training an XGBoost Regressor'></a>

In [ ]:
# create a baseline
booster = xgb.XGBRegressor()

Some of the important hyperparameters to tune an XGBoost are:
- `n_estimators` $\;\;\;\;\;$ = Number of trees one wants to build.
- `learning_rate` $\;\;\;\;$= Rate at which our model learns patterns in data. After every round, it shrinks the feature weights to reach the best optimum.
- `max_depth` $\;\;\;\;\;\;\;\;\;\;\;$= Determines how deeply each tree is allowed to grow during any boosting round.
- `colsample_bytree` = Percentage of features used per tree. 
- `gamma` $\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$= Specifies the minimum loss reduction required to make a split.

In [ ]:
from sklearn.model_selection import GridSearchCV

# create Grid
param_grid = {'n_estimators': [100, 150, 200],
              'learning_rate': [0.01, 0.05, 0.1], 
              'max_depth': [3, 4, 5, 6, 7],
              'colsample_bytree': [0.6, 0.7, 1],
              'gamma': [0.0, 0.1, 0.2]}

# instantiate the tuned random forest
booster_grid_search = GridSearchCV(booster, param_grid, cv=3, n_jobs=-1)

# train the tuned random forest
booster_grid_search.fit(X_train, y_train)

# print best estimator parameters found during the grid search
print(booster_grid_search.best_params_)

In [ ]:
# instantiate xgboost with best parameters
booster = xgb.XGBRegressor(colsample_bytree=0.7, gamma=0.2, learning_rate=0.1, 
                           max_depth=6, n_estimators=200, random_state=4)

# train
booster.fit(X_train, y_train)

# predict
y_pred_train = booster.predict(X_train)
y_pred_test = booster.predict(X_test)

In [ ]:
RMSE = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f"RMSE: {round(RMSE, 4)}")

In [ ]:
r2 = r2_score(y_test, y_pred_test)
r2
print(f"r2: {round(r2, 4)}")

#### 4.4. Cross Validation
<a id='4.4. Cross Validation'></a>

In order to build more robust models, it is common to conduct a k-fold cross validation where all the entries in the original training dataset are used for both training and validation. XGBoost supports k-fold cross validation via the cv method. All we have to do is specify the `nfolds` parameter, which is the number of cross validation rounds you want to build. 

Also, it supports many other parameters:
- `num_boost_round` $\;\;\;\;\;\;\;\;$ = Specifies the number of trees to build (analogous to n_estimators).
- `metrics` $\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$ = Specifies the evaluation metrics to be checked during CV.
- `as_pandas` $\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;$= Returns the results in a decent pandas DataFrame.
- `early_stopping_rounds` = Finishes model training early if the hold-out metric does not improve for a given number of rounds. 

We will have to first convert the dataset into an optimized data structure called DMatrix so that XGBoost's cross validation method is supported.

In [ ]:
xg_train = xgb.DMatrix(data=X_train, label=y_train)

In [ ]:
params = {'colsample_bytree':0.6, 'gamma':0.2, 'learning_rate':0.05, 'max_depth':6}

cv_results = xgb.cv(dtrain=xg_train, params=params, nfold=3,
                    num_boost_round=200, early_stopping_rounds=10, 
                    metrics="rmse", as_pandas=True)

In [ ]:
cv_results.head()

In [ ]:
cv_results.tail()

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 5. Interpreting the Data 
<a id='5. Interpreting the Data'></a>

We can see that our average error (RMSE) in the initial XGBoost is around 22€, which improves to 17.5€ by cross validation. Given the fact that after cleaning up the price column, 50% of our lodgings cost only up to 45€ and 75% of our lodgings up to 70€ - even the improved standard deviation of 17€ is quite a massive inaccuracy that doesn't help much in recommending a price.

It turns out that the price is dependent not only on geography, size, and features. It stands to reason that 
- the quality of presentation (e.g. pictures), 
- availability, 
- the number and content of reviews, 
- communication (e.g. acceptance rate, host response time) or 
- status ((whether or not the host is a super host)) 

might have a substantial influence too. But the purpose of this analysis was to recommend a price to a "rookie" without any reviews or status. With this in mind, we might say that we can't recommend an exact price, but rather a range..

The next step (and maybe an idea for the reader) would be to start all over again and include the features mentioned above to try to find out if accuracy improves. That might help a beginner on Airbnb better know what price to aim for.

With what we have done here, we have explained 71% of the variance (R^2) with the most important accommodation features, as pictured below:

In [ ]:
# plot the important features
feat_importances = pd.Series(booster.feature_importances_, index=features_recoded.columns)
feat_importances.nlargest(15).sort_values().plot(kind='barh', color='darkgrey', figsize=(10,5))
plt.xlabel('Relative Feature Importance with XGBoost');

As we see, the **most important features are size, distance, and cleaning fee**, which account for approximately 45% of the daily price. Other top features are the number of people the apartment accommodates, other fees such as security deposit or the price for extra people, minimum night stay and bathrooms.

*Back to: <a href='#Table of contents'> Table of contents</a>*
### 6. Appendix 
<a id='6. Appendix'></a>

All resources used in this notebook are listed below.

Data
- http://insideairbnb.com/get-the-data.html

Imputing missing values with Linear Regression
- https://towardsdatascience.com/the-tale-of-missing-values-in-python-c96beb0e8a9d

XGBoost
- https://www.datacamp.com/community/tutorials/xgboost-in-python
- https://www.kaggle.com/marcelo06/cross-validation-with-xgboost-python

Geocoding
- https://pypi.org/project/geopy/

Centroid of Berlin (in German!)
- https://www.tagesspiegel.de/berlin/bezirke/friedrichshain-kreuzberg/reise-zum-mittelpunkt-berlins-am-ruhepol/9837502.html

Visualizations
- https://python-graph-gallery.com
- https://www.kaggle.com/kostyabahshetsyan/boston-airbnb-visualization

Inspiration for Analysis
- https://towardsdatascience.com/digging-into-airbnb-data-reviews-sentiments-superhosts-and-prices-prediction-part1-6c80ccb26c6a
- https://www.kaggle.com/ibjohnsson/predicting-listing-prices
- https://www.kaggle.com/mathvv/prediction-on-house-prices-xgboost-tutorial
- https://towardsdatascience.com/improving-airbnb-yield-prediction-with-text-mining-9472c0181731
- https://github.com/joaeechew/airbnb_nlp/blob/master/Capstone%20Project.ipynb